**10. MLOps: Production Deployment & Monitoring**

 Step 1: Fixed & Enhanced SeldonDeployment YAML
 

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: emission-anomaly-detector
  labels:
    app: emission-anomaly-detector
  annotations:
    prometheus.io/scrape: "true"
    prometheus.io/path: "/metrics"
    prometheus.io/port: "8080"
spec:
  name: detector
  predictors:
    - name: main-predictor
      replicas: 1
      graph:
        name: anomaly-model
        type: MODEL
        endpoint:
          type: REST
      componentSpecs:
        - spec:
            containers:
              - name: anomaly-model
                image: my-anomaly-model:0.1
                imagePullPolicy: IfNotPresent
                ports:
                  - containerPort: 8080
                    name: http
                resources:
                  requests:
                    cpu: "100m"
                    memory: "128Mi"
                  limits:
                    cpu: "500m"
                    memory: "512Mi"


Explanation of Fixes: 
| Fix                         | What Was Done                                              |
| --------------------------- | ---------------------------------------------------------- |
| **Resource limits**       | Added `cpu`/`memory` under `resources`                     |
| **Port definition**       | Declared `containerPort: 8080` for REST access             |
| **Prometheus monitoring** | Added scraping annotations in `metadata.annotations`       |
| **Model metadata**        | Used `imagePullPolicy` and `name` to standardize container |


Step 2: Designing Data Drift Detection

**Goal:**

Detect when the incoming data in production differs significantly from the training data — helping flag model degradation over time.

 Pseudo-code Example (KS Test for Drift)

In [2]:
from scipy.stats import ks_2samp

def detect_drift(train_data, live_data, threshold=0.1):
    drift_results = {}
    for column in train_data.columns:
        stat, p_value = ks_2samp(train_data[column], live_data[column])
        drift_results[column] = {
            "p_value": p_value,
            "drift_detected": p_value < threshold
        }
    return drift_results
